In [ ]:
# importando o conjunto de dados:  breast_cancer_formated.csv 
from google.colab import files
data = files.upload()

Saving breast_cancer_formated.csv to breast_cancer_formated.csv


In [ ]:
 # lendo os dados e criando o dataFrame
import pandas as pd
import io

df = pd.read_csv(io.StringIO(data['breast_cancer_formated.csv'].decode('utf-8')))

In [ ]:
# renomeando as colunas
df=df.rename(columns = {'5.000000000000000000e+00':'Clump Thickness','1.000000000000000000e+00':'Uniformity of Cell Size','1.000000000000000000e+00.1':'Uniformity of Cell Shape:','1.000000000000000000e+00.2':'Marginal Adhesion','2.000000000000000000e+00': 'Single Epithelial Cell Size','1.000000000000000000e+00.3': 'Bare Nucle','3.000000000000000000e+00': 'Bland Chromatin','1.000000000000000000e+00.4':'Normal Nucleoli', '1.000000000000000000e+00.5':'Mitoses', '0.000000000000000000e+00':'Classes'})
df

In [ ]:
#X features/características e y = target/rotulos( quem o modelo quer prevê)
# linhas e colunas das features e target as classes.
X = df.iloc[:,:-1].values
y = df.iloc[:,9].values

print(X)
print(y)



In [ ]:
from sklearn.model_selection import train_test_split
# divide os dados em dois conjuntos (treino e teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)


print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

In [ ]:
# importa o MLPClassifier da biblioteca sklearn
from sklearn.neural_network import MLPClassifier
# instancia um objeto da tecnica MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=600, activation='relu',learning_rate_init=0.01)
# treina um modelo de classificacao
mlp.fit(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(mlp.loss_curve_,label="treino")

plt.title("Curva do erro de treinamento \n learning_rate 0.01 \n max_iter 600")
plt.xlabel("Iterações")
plt.ylabel("Erro")
plt.legend()

In [ ]:
# comparação do resultado predito com o esperado
y_pred=mlp.predict(X_test)

y_true = y_test

print("Classe predita:   ", y_pred)
print("Classe verdadeira:", y_true)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

In [ ]:
#tn = verdadeiro negativo
#fp = falso positivo 
#fn = falso negativo 
#tp = verdadeiro positivo  

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
(tn, fp, fn, tp)

(147, 6, 3, 70)

In [ ]:

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
acuSemGridS = acc
[acc, prec, f1, recall]

[0.9601769911504425, 0.9210526315789473, 0.9395973154362416, 0.958904109589041]

In [ ]:
# precisão do resultado
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.98      0.96      0.97       153
         1.0       0.92      0.96      0.94        73

    accuracy                           0.96       226
   macro avg       0.95      0.96      0.95       226
weighted avg       0.96      0.96      0.96       226



In [ ]:
#importando o GridSearchCV
from sklearn.model_selection import GridSearchCV
mlp2 = MLPClassifier(random_state=1)

parameters = { 'hidden_layer_sizes':[(100,100),(10,100,120),(100,120,180)],'learning_rate_init': [0.01,0.1,0.001] ,'activation': ["identity","tanh", "relu"],'max_iter': [100,300,500]}

In [ ]:
grid = GridSearchCV(mlp2,parameters)
grid.fit(X_train, y_train)

In [ ]:
#tabela com as 5 melhores combinações encontradas pela GrindSearchCV

pd.DataFrame(grid.cv_results_).sort_values(by=['rank_test_score'])[['params','rank_test_score','mean_test_score']].head(5)

,params,rank_test_score,mean_test_score
24,"{'activation': 'identity', 'hidden_layer_sizes...",1,0.971524
27,"{'activation': 'tanh', 'hidden_layer_sizes': (...",2,0.971500
26,"{'activation': 'identity', 'hidden_layer_sizes...",3,0.969350
25,"{'activation': 'identity', 'hidden_layer_sizes...",3,0.969350
15,"{'activation': 'identity', 'hidden_layer_sizes...",3,0.969350


In [ ]:
# Imprime os parâmetros que produziram o ".best_score_".
grid.best_params_

{'activation': 'identity',
 'hidden_layer_sizes': (10, 100, 120),
 'learning_rate_init': 0.001,
 'max_iter': 100}

In [ ]:
# Imprimindo o score.
grid.best_score_

0.9715241280458672

In [ ]:
from statistics import mean
l_acc = []
l_prec = []
l_f1 = []
l_recall = []

In [ ]:
# validação cruzada com k-fold com o melhor modelo encontrado pelo GridSearchCV
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
mlpBest = MLPClassifier(hidden_layer_sizes = (10,100,120),random_state=1, max_iter=100, activation='identity',learning_rate_init = 0.001)

for train_indices, test_indices in kf.split(X):
    mlpBest.fit(X[train_indices], y[train_indices])
    y_pred=mlp.predict(X[test_indices])
    acc = accuracy_score(y[test_indices], y_pred)
    prec = precision_score(y[test_indices], y_pred, average='macro')
    f1 = f1_score(y[test_indices], y_pred, average='macro')
    recall = recall_score(y[test_indices], y_pred, average='macro')
    l_acc.append(acc)
    l_prec.append(prec)
    l_f1.append(f1)
    l_recall.append(recall)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
#media das metricas
print("A média da  acc é ", round(mean(l_acc),2))
print("A média da precisão é ", round(mean(l_prec),2))
print("A média f1 é ", round(mean(l_f1),2))
print("A média do recall é ", round(mean(l_recall),2))


A média da  acc é  0.98
A média da precisão é  0.98
A média f1 é  0.98
A média do recall é  0.98


In [ ]:
#comparando os resultados da acuracia com e sem o uso do GridSearchCV
acCG = round(mean(l_acc),2)
acSG = round(acuSemGridS,2)
table = pd.DataFrame({"MLB": [acSG, acCG]} ,index=['Sem GridSearchCV','Com GridSearchCV'])
table.style.set_caption('Tabela de Comparação das acurácia')



,MLB
Sem GridSearchCV,0.960000
Com GridSearchCV,0.980000
